In [2]:
import pandas as pd
import re
import nltk
import numpy as np

from nltk.corpus import sentiwordnet as swn

In [7]:
# 처음에 nltk 가 import 안될 경우는 패키지가 제대로 다운이 되지 않은 것 때문이므로 아래 명령어로 다운로드
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
## 나눠서 수집한 기사 결합
정상기업뉴스1 = pd.read_csv('./datasets/정상기업번역파일.csv')
정상기업뉴스1.drop('뉴스기사본문전처리', axis=1, inplace=True)

정상기업뉴스2 = pd.read_csv('./datasets/정상기업번역파일추가.csv')
정상기업뉴스2.drop('뉴스기사본문전처리', axis=1, inplace=True)

정상기업뉴스 = pd.concat([정상기업뉴스1, 정상기업뉴스2], axis=0)
정상기업뉴스.reset_index(inplace=True, drop=True)

정상기업뉴스['기사본문번역'] = [re.sub('[가-힣]', '', s) for s in 정상기업뉴스['기사본문번역']]
## 정상기업 표본에서는 년도만 다르고 이름이 같은 기업이 있기 때문에 Groupby할때 기사발행년도와 기업이름 필요
정상기업뉴스['기사발행일'] = 정상기업뉴스['기사발행일'].apply(lambda x: x[:4])

In [10]:
## 기사 토큰화 및 품사 태깅
정상기업뉴스['기사본문번역토큰'] = 정상기업뉴스['기사본문번역'].apply(lambda x: nltk.word_tokenize(x))
정상기업뉴스['기사본문번역토큰'] = 정상기업뉴스['기사본문번역토큰'].apply(lambda x: nltk.pos_tag(x))

In [11]:
## 문장에서 명사 동사 형용사 부사에 한하여 부정 긍정지수 계산
def word_sentiment_calculator(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    else:
        return (0,0)
    
    for syn in syn_set:
        pos_score += syn.pos_score()
        neg_score += syn.neg_score()
    return (pos_score/len(syn_set), neg_score/len(syn_set))

## 출처 : https://github.com/buomsoo-kim/Sentiment-Analysis-with-Python/blob/master/source%20code/session%204%20-%20Text%20analysis/iphynb%20files/%5B4-1%5D%20SentiwordNet.ipynb

In [13]:
## 문장의 감성 지수 계산하기 위한 함수
def sentence_sentiment_calculator(sent):
    tokens =  nltk.word_tokenize(sent)
    pos_tags = nltk.pos_tag(tokens)
    
    pos_score = 0
    neg_score = 0
    for word, tag in pos_tags:
        pos_score += word_sentiment_calculator(word, tag)[0]
        neg_score += word_sentiment_calculator(word, tag)[1]
    return (pos_score, neg_score)

## 출처 : https://github.com/buomsoo-kim/Sentiment-Analysis-with-Python/blob/master/source%20code/session%204%20-%20Text%20analysis/iphynb%20files/%5B4-1%5D%20SentiwordNet.ipynb

In [15]:
## 긍정일경우 0 부정일경우 1
predicted = []

for i in range(len(정상기업뉴스)):
    scores = sentence_sentiment_calculator(정상기업뉴스.기사본문번역[i])

    #긍정점수 >= 부정점수 일 경우 긍정
    if scores[0] >= scores[1]:
        predicted.append(0)
    #부정점수 < 긍정점수 일 경우 부정
    else:
        predicted.append(1)

정상기업뉴스['긍정부정분류'] = predicted

In [18]:
## Groupby활용하여 기업별 부정기사수, 전체기사수를 추출하고 부정기사비율로 계산
## 위에서 언급한 것처럼 정상기업은 중복되는 경우가 있어 Groupby할때 기사발행일까지 필요
전체기사수 = pd.DataFrame(정상기업뉴스.groupby(['기업', '기사발행일']).count()['긍정부정분류'])
전체기사수.reset_index(drop = False, inplace = True)
전체기사수.columns = ['기업', '기사발행일', '전체기사수']

부정기사수 = pd.DataFrame(정상기업뉴스[정상기업뉴스.긍정부정분류==1].groupby(['기업', '기사발행일']).count()['긍정부정분류'])
부정기사수.reset_index(drop = False, inplace = True)
부정기사수.columns = ['기업', '기사발행일', '부정기사수']

정상기업 = pd.merge(전체기사수, 부정기사수, on=['기업', '기사발행일'], how='left')
정상기업['부정기사비율'] = (정상기업['부정기사수'] / 정상기업['전체기사수']) * 100
정상기업.fillna(0, inplace=True)
정상기업['기사발행일'] = 정상기업['기사발행일'].astype(int)
정상기업['기사발행일'] = 정상기업['기사발행일'].apply(lambda x: x-2)
정상기업.drop(['전체기사수', '부정기사수'], axis=1, inplace=True)
정상기업.columns = ['회사명', '회계년도', '부정기사비율']
정상기업

,회사명,회계년도,부정기사비율
0,경창산업,2011,8.064516
1,고영테크놀러지,2009,4.255319
2,광림,2014,25.454545
3,금화피에스시,2011,28.169014
4,나스미디어,2015,11.594203
...,...,...,...
119,하이록코리아,2010,1.408451
120,한국정보통신,2013,9.090909
121,해성산업,2011,14.285714
122,홈캐스트,2014,23.913043


In [24]:
## 정상기업 재무비율을 가져와서 부정기사비율과 결합하여 저장
정상기업재무비율 = pd.read_csv('../Step1-2_재무비율_모델링/datasets/재무비율.csv')
정상기업재무비율 = 정상기업재무비율[정상기업재무비율.부실기업여부==0]
정상기업_감성분석 = pd.merge(정상기업재무비율, 정상기업, how='left', on=['회사명', '회계년도'])
정상기업_감성분석.fillna(0, inplace=True)
정상기업_감성분석.to_csv('../Step5-2_감성분석_모델링/datasets/정상기업_부정기사비율_데이터.csv', index=None, encoding='utf-8-sig')
